In [3]:
from specpy import *
from Util.coordinates import *
from Util.datastructures import *
from Util.tile_util import *
from Util.imspector_util import *
from Util.dot_detection.Fiji_coordinate_detection import *
from Util.coordinate_util import *
from Util.dot_detection.Fiji_dummy import *
from Util.dot_detection.Fiji_coordinate_detection import read_coords
import pprint
from Util import datastructures, imspector_util
import time
from matplotlib import pyplot as plt
from skimage.feature import blob_dog, blob_log, blob_doh
from scipy import ndimage, spatial, stats
from spot_util import *
%matplotlib inline

In [5]:
def detect_blobs(img, sigmas, threshold):
    img = skimage.util.img_as_float(img)
    img = img / np.max(img)
    logimg = -ndimage.gaussian_laplace(img, sigmas)
    peaks = skimage.feature.peak_local_max(logimg, threshold_abs=threshold)
    return peaks

In [4]:
def pair_finder(ms, pix_sig=3, threshold=0.01):

    stack1 = ms.stack(0).data()[0,:,:,:]
    stack2 = ms.stack(1).data()[0,:,:,:]

    sig = pix_sig / np.sqrt(2)
    dets1 = detect_blobs(stack1, [sig, sig, sig], threshold)    
    dets1=list(dets1)
    dets2 = detect_blobs(stack2, [sig, sig, sig], threshold)    
    dets2=list(dets2)

    if (len(dets1) == 0 or len(dets2) == 0):
        return []
    
    kd1 = spatial.KDTree(dets1)
    kd2 = spatial.KDTree(dets2)

    cleanup_kdtree(stack1, kd1, dets1, 3)
    cleanup_kdtree(stack2, kd2, dets2, 3)

    res = []
    for p in find_pairs(kd2, dets1, 5):
        res.append(list(p))

    return res

In [6]:
def cleanup_kdtree(img, kdt, dets, dist):
    notdone = True
    while notdone:
        notdone = False
        for i in range(len(dets)):
            nn = kdt.query(dets[i],k=2, distance_upper_bound=dist)
            if (not np.isinf(nn[0][1])) and nn[0][1] > 0:
                ineighbor = nn[1][1]
                if img[tuple(dets[i])] < img[tuple(dets[ineighbor])]:
                    dets.pop(i)
                else:
                    dets.pop(ineighbor)
                notdone = True
                    
                kdt = spatial.KDTree(dets)
                break
        
def find_pairs(kdt2, dets1, dist):
    res = []
    for d in dets1:
        nn = kdt2.query(d, distance_upper_bound=dist)
        if not np.isinf(nn[0]):
            # dets were in zyx -> turn to xyz
            res.append(d[-1::-1])
    return res

In [7]:
def mean_along_axis(img, axis):
    axes = tuple([ax for ax in range(len(img.shape)) if ax != axis])
    #print(axes)
    profile = np.mean(img, axes)
    return profile

def focus_in_stack(img, pixsize_z):
    profile = mean_along_axis(img, 1)
    smoothprofile = ndimage.gaussian_filter1d(profile, 3, mode='constant')
    tmax = np.argmax(smoothprofile)
    pix_d = tmax - (len(profile) / 2)
    return pix_d * pixsize_z

focus_in_stack(ms.stack(0).data(), 400e-09)

NameError: name 'ms' is not defined

In [63]:
see = Settings()
co = Coordinates()

In [2]:
# make results dir
f = 'C:/Users//RESOLFT/Desktop/AUTOMATION/82_2min/'
d = os.path.dirname(f)
if not os.path.exists(d):
    os.makedirs(d)

im = Imspector()

see = Settings()
see.load_from_file('C:/Users//RESOLFT/Desktop/ov.json')
coords = generate_grid_oop([0,0],[2e-3, 2e-3], [5e-5, 5e-5])
for c in coords:
    c.set_fov_len([5e-05, 5e-05, 1.2e-5])
   

#keep track of maximum z -> use that as next midplane
zpos = 0.0;
    
nm = datastructures.NameManagement(d)
nm.add_counter('field')
nm.add_counter('sted')

for c in coords:
    
    offset_old = c.get_scan_offset()
    offset_old[2] = zpos
    c.set_offset_coords(offset_old)
    
    see.set_to_coordinates(c)
    
    
    name = nm.get_next_image_name('field')
    acquire_measurement_dummy(im, see)
    ms = im.active_measurement()
    
    sites = pair_finder(ms)
    
    co = c.copy()
    
    corner = middle2corner(c.get_scan_offset(), c.get_fov_len())
    pixelsd = get_pixel_size(ms)
    actual_coords = corner2spot(corner, sites, pixelsd)
    
    # update focus
    zpos += focus_in_stack(ms.stack(0).data(), pixelsd[2])    
    
    ms.save_as(name)
    im.close(ms)
    
    print(sites)
    
    see2 = Settings()
    see2.load_from_file('C:/Users//RESOLFT/Desktop/det.json')
    
    nm.reset_counter('sted')
    
    for i in actual_coords:
        co.set_fov_len([3e-06, 3e-06, 1e-06])
        co.set_offset_coords(i)
        see2.set_to_coordinates(co)
        
        name = nm.get_next_image_name('sted')
        acquire_measurement_dummy(im, see2)
        
        ms = im.active_measurement()
        ms.save_as(name)
        im.close(ms)

KeyboardInterrupt: 

In [20]:
stack1 = ms.stack(0).data()[0,:,:,:]
stack2 = ms.stack(1).data()[0,:,:,:]

sig = 3 / np.sqrt(2)
dets1 = detect_blobs(stack1, [sig, sig, sig], 0.002)    
dets1=list(dets1)
dets2 = detect_blobs(stack2, [sig, sig, sig], 0.002)    
dets2=list(dets2)

In [17]:
actual_coords

[[-1.54e-05, -1.615e-05, 1.9999999999999995e-06],
 [1.1449999999999997e-05, 7.549999999999997e-06, 4.8e-06]]

In [20]:
co

In [22]:
actual_coords

[[0.0006972, -1.5250000000000001e-05, 3.199999999999999e-06],
 [0.00072435, 8.75e-06, 4.799999999999999e-06],
 [0.0006774, -1.2100000000000001e-05, 7.2e-06],
 [0.0006852000000000001, -5.500000000000032e-07, 7.599999999999999e-06]]